In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import evo.core.trajectory as et
import evo.core.metrics as em
from evo.tools import file_interface, plot
from evo.tools.settings import SETTINGS
import os
import copy

# Load environment variables for file paths
BAGFILES_PATH = "/mnt/storage/bags/shellbe_2024-02-21/final/shellbeLabCalibration"
gt_file = f"{BAGFILES_PATH}/reference/totalStation_shellbeLabCalibration_2025-02-25-16-52-55_reference.csv"
est_file = f"{BAGFILES_PATH}/evaluation_output/estimated_trajectory.csv"

traj_ref = file_interface.read_tum_trajectory_file(gt_file)
traj_est = file_interface.read_tum_trajectory_file(est_file)


In [ ]:
from evo.core import sync
max_diff = 0.05

traj_ref_synced, traj_est_synced = sync.associate_trajectories(traj_ref, traj_est, max_diff)

traj_est_aligned = copy.deepcopy(traj_est_synced)
traj_ref_aligned = copy.deepcopy(traj_ref_synced)
traj_est_aligned.align(traj_ref_synced, correct_scale=False, correct_only_scale=False)

In [ ]:
# set both trajectories orientations to identity
from evo.core.trajectory import PoseTrajectory3D
def set_identity_orientations(traj):
    identity_orientations = np.zeros((len(traj.positions_xyz), 4))
    identity_orientations[:, 0] = 1
    return PoseTrajectory3D(positions_xyz=traj.positions_xyz, orientations_quat_wxyz=identity_orientations, timestamps=traj.timestamps)

traj_ref_aligned = set_identity_orientations(traj_ref_synced)
traj_est_aligned = set_identity_orientations(traj_est_aligned)

data = (traj_ref_aligned, traj_est_aligned)

In [ ]:
from evo.core import metrics
# compute the ATE
pose_relation = metrics.PoseRelation.translation_part

ape_metric = metrics.APE(pose_relation)
ape_metric.process_data(data)
ape_results = ape_metric.get_statistic(metrics.StatisticsType.rmse)
ape_stat = ape_metric.get_all_statistics()

In [ ]:
def compute_RPE(data, delta_meters):
    delta = delta_meters
    delta_unit = metrics.Unit.meters

    rpe_metric = metrics.RPE(pose_relation, delta, delta_unit, all_pairs=True)
    try:
        rpe_metric.process_data(data)
    except:
        return None
    rpe_stat = rpe_metric.get_all_statistics()

    return rpe_stat

def get_rpe_info(rpe_stat):
    return f"{rpe_stat['rmse']} +- {rpe_stat['std']}  ({rpe_stat['min']}, {rpe_stat['max']})"


SMALL_DELTAS = [1, 2, 5, 10, 20, 50, 100]
delta_meters_big = [100, 200, 500, 800]

def compute_rpe_set(data, delta_meters):
    results = {
        
    }
    for delta in delta_meters:
        result = compute_RPE(data, delta)
        if result is not None:
            results[delta] = result
        else:
            print (f"Error processing delta {delta}, skipping")
    return results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import yaml
from mpl_toolkits.mplot3d import Axes3D

def compute_rpe_table(rpe_results):
    rpe_table = []
    average_relative_rpe = []
    for delta, stats in rpe_results.items():
        relative_rpe = (stats['rmse'] / delta) * 100  # Convert to percentage
        average_relative_rpe.append(relative_rpe)
        rpe_table.append([
            f"{delta}m", 
            f"RMSE: {relative_rpe:.2f}%\nSTD: {(stats['std']/delta) * 100:.2f}%\nMIN: {(stats['min']/delta) * 100:.2f}%\nMAX: {(stats['max']/delta) * 100:.2f}%",
            f"RMSE: {stats['rmse']:.3f} [m]\nSTD: {stats['std']:.3f} [m]\nMIN: {stats['min']:.3f} [m]\nMAX: {stats['max']:.3f} [m]"
        ])
    return rpe_table, float(np.mean(average_relative_rpe))

def set_equal_aspect_3d(ax, traj):
    x_limits = [np.min(traj[:, 0]), np.max(traj[:, 0])]
    y_limits = [np.min(traj[:, 1]), np.max(traj[:, 1])]
    z_limits = [np.min(traj[:, 2]), np.max(traj[:, 2])]
    max_range = max(np.ptp(x_limits), np.ptp(y_limits), np.ptp(z_limits))
    mid = lambda lims: np.mean(lims)
    ax.set_xlim(mid(x_limits) - max_range/2, mid(x_limits) + max_range/2)
    ax.set_ylim(mid(y_limits) - max_range/2, mid(y_limits) + max_range/2)
    ax.set_zlim(mid(z_limits) - max_range/2, mid(z_limits) + max_range/2)

def save_rpe_yaml(filename, avg_rpe, ate_rmse, rpe_results):
    rpe_data = {
        'results': {
            'rpe_avg_rmse_percentage': float(avg_rpe),
            'ate_rmse_meters': float(ate_rmse)
        },
        'rpe_details': {
            f"{delta}m": {
                'rmse_meters': float(stats['rmse']),
                'std_meters': float(stats['std']),
                'min_meters': float(stats['min']),
                'max_meters': float(stats['max'])
            } for delta, stats in rpe_results.items()
        }
    }
    with open(filename, 'w') as file:
        yaml.dump(rpe_data, file)

# Compute RPE results
rpe_results_small = compute_rpe_set(data, SMALL_DELTAS)
rpe_table, avg_relative_rpe = compute_rpe_table(rpe_results_small)
ate_rmse = float(np.sqrt(np.mean([s['rmse']**2 for s in rpe_results_small.values()])))

# Save YAML results
save_rpe_yaml(f"{BAGFILES_PATH}/evaluation_output/trajectory_analysis.yaml", avg_relative_rpe, ate_rmse, rpe_results_small)

# Save trajectory plots and statistics
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# XY Plot
ax_2d = axs[0, 0]
ax_2d.plot(traj_ref_aligned.positions_xyz[:, 0], traj_ref_aligned.positions_xyz[:, 1], label="Reference", linestyle='-', marker='o', markersize=2)
ax_2d.plot(traj_est_aligned.positions_xyz[:, 0], traj_est_aligned.positions_xyz[:, 1], label="Estimated", linestyle='-', marker='x', markersize=2)
ax_2d.set_xlabel("X Position (m)")
ax_2d.set_ylabel("Y Position (m)")
ax_2d.set_title("XY Trajectory Plot")
ax_2d.legend()
ax_2d.grid()
ax_2d.set_aspect('equal', adjustable='datalim')

# Summary Table
axs[0, 1].axis('tight')
axs[0, 1].axis('off')
axs[0, 1].set_title("SUMMARY METRICS", fontsize=12, fontweight='bold')
sum_table = axs[0, 1].table(cellText=[[f"{avg_relative_rpe:.2f} %", f"{ate_rmse:.3f} [m]"]],
                             colLabels=["AVG RMSE RPE (%)", "ATE RMSE (m)"],
                             loc='center', cellLoc='center')
sum_table.auto_set_font_size(False)
sum_table.set_fontsize(10)
sum_table.scale(1.0, 2.4)

# 3D Plot
ax_3d = axs[1, 0]
ax_3d = fig.add_subplot(2, 2, 3, projection='3d')
ax_3d.plot(traj_ref_aligned.positions_xyz[:, 0], traj_ref_aligned.positions_xyz[:, 1], traj_ref_aligned.positions_xyz[:, 2], label="Reference")
ax_3d.plot(traj_est_aligned.positions_xyz[:, 0], traj_est_aligned.positions_xyz[:, 1], traj_est_aligned.positions_xyz[:, 2], label="Estimated")
ax_3d.set_xlabel("X Position (m)")
ax_3d.set_ylabel("Y Position (m)")
ax_3d.set_zlabel("Z Position (m)")
ax_3d.set_title("3D Trajectory Plot")
ax_3d.legend()
ax_3d.grid()
set_equal_aspect_3d(ax_3d, traj_ref_aligned.positions_xyz)
BAGFILES_PATH
axs[1, 1].axis('off')
axs[1, 1].set_title("RPE DETAILS", fontsize=12, fontweight='bold')
rpe_table_plot = axs[1, 1].table(cellText=rpe_table,
                                  colLabels=["Delta", "Relative RPE", "Absolute RPE [m]"],
                                  loc='center', cellLoc='center')
rpe_table_plot.auto_set_font_size(False)
rpe_table_plot.set_fontsize(9)
rpe_table_plot.scale(1.0, 4.0)

# Save the figure
plt.tight_layout()
plt.savefig(f"{BAGFILES_PATH}/evaluation_output/trajectory_analysis.pdf", format="pdf", dpi=300)
plt.close()